# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298288055139                   -0.85    5.0
  2   -8.300176839277       -2.72       -1.25    1.0
  3   -8.300444807035       -3.57       -1.88    4.1
  4   -8.300462760205       -4.75       -2.75    1.5
  5   -8.300464206782       -5.84       -3.02    6.0
  6   -8.300464395866       -6.72       -3.18    2.1
  7   -8.300464515732       -6.92       -3.32    1.4
  8   -8.300464579793       -7.19       -3.46    1.0
  9   -8.300464628319       -7.31       -3.67    1.0
 10   -8.300464632416       -8.39       -3.73   10.9
 11   -8.300464640492       -8.09       -3.92    1.1
 12   -8.300464643925       -8.46       -4.42    1.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64187832683                   -0.70    6.2
  2   -16.67828943620       -1.44       -1.14    1.4
  3   -16.67922078889       -3.03       -1.88    2.4
  4   -16.67928176749       -4.21       -2.76    3.9
  5   -16.67928609999       -5.36       -3.12    5.1
  6   -16.67928621701       -6.93       -3.55    1.1
  7   -16.67928622279       -8.24       -4.08    1.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32737151673                   -0.56    8.0
  2   -33.33453741964       -2.14       -1.00    1.2
  3   -33.33602577451       -2.83       -1.72    5.2
  4   -33.33615508390       -3.89       -2.63    8.0
  5   -33.33681998911       -3.18       -2.28    6.0
  6   -33.33685118651       -4.51       -2.35    2.5
  7   -33.33694375676       -4.03       -3.47    3.6
  8   -33.33694391908       -6.79       -3.74    6.1
  9   -33.33694383911   +   -7.10       -3.87    1.8
 10   -33.33694392709       -7.06       -5.23    2.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298447716453                   -0.85    5.0
  2   -8.300267415856       -2.74       -1.59    1.5
  3   -8.300451007934       -3.74       -2.56    4.8
  4   -8.300384293306   +   -4.18       -2.30   12.8
  5   -8.300464367784       -4.10       -3.57    3.0
  6   -8.300464589922       -6.65       -3.94    2.4
  7   -8.300464642572       -7.28       -4.71    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32602206733                   -0.56    7.2
  2   -33.30627668940   +   -1.70       -1.26    1.5
  3   -5.541855292464   +    1.44       -0.34    8.4
  4   -33.33352316590        1.44       -1.91    6.6
  5   -33.23220807037   +   -0.99       -1.32    5.2
  6   -33.08334944361   +   -0.83       -1.36    5.0
  7   -33.28430087996       -0.70       -1.69    5.1
  8   -33.33585269760       -1.29       -2.42    4.0
  9   -33.33668270568       -3.08       -2.60    2.4
 10   -33.33644827641   +   -3.63       -2.59    2.5
 11   -33.33690545586       -3.34       -3.10    2.2
 12   -33.33692370515       -4.74       -3.26    3.4
 13   -33.33694345769       -4.70       -3.73    3.1
 14   -33.33694382302       -6.44       -4.00    3.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.